## <span style="color:#0b486b">Naive Bayes Classifier</span> 


Naive Bayes is one of the most practical classification machine learning algorithms. 

* fast
* good performance
* simple yet very effective
* robust to irrelative features

So why is it called naive?

Because it does not consider the dependency between features and assume all features are independent of each other which is not the case in reality. This is a naive assumption, hence the name.

The accuracy is very good although this naive assumption. A famous example of NB usage is spam filtering.

---
### Example1

We assume we have collected the below data for the past 5 days. Based on this data, can we predict if our subject will play in a setting like:

    outlook  = overcast
    temp     = hot
    humidity = normal
    windy    = no
    
<table border=1>
  <tr>
    <th>Outlook</th>
    <th>Temperature</th>
    <th>Humidity</th>
    <th>Windy</th>
    <th>Play</th>
  </tr>
  <tr>
    <td>sunny</td>
    <td>hot</td>
    <td>normal</td>
    <td>no</td>
    <td>yes</td>
  </tr>
  <tr>
    <td>overcast</td>
    <td>mild</td>
    <td>normal</td>
    <td>no</td>
    <td>yes</td>
  </tr>
    <tr>
    <td>rainy</td>
    <td>cool</td>
    <td>high</td>
    <td>yes</td>
    <td>no</td>
  </tr>
    <tr>
    <td>sunny</td>
    <td>mild</td>
    <td>normal</td>
    <td>yes</td>
    <td>no</td>
  </tr>
    <tr>
    <td>overcast</td>
    <td>hot</td>
    <td>high</td>
    <td>no</td>
    <td>no</td>
  </tr>
</table>
    


First we have to find a representation for our data. We can construct a dictionary to convert stings into numbers and then save them in a dataframe. 

    outlook: sunny=0, overcast=1, rainy=2
    temp: hot=0, mild=1, cool=2
    humidity: normal=0, high=1
    wind: no=0, yes=1
    play: np=0, yes=1

In [3]:
import numpy as np
import pandas as pd

In [4]:
data = {
 'outlook': [0, 1, 2, 0, 1],
 'temp' : [0, 1, 2, 1, 0],
 'humid' : [0, 0, 1, 0, 1],
 'wind' : [0, 0, 1, 1, 0],
 'play' : [1, 1, 0, 0, 0,] 
}

df = pd.DataFrame(data)

Now we use Bayes rule to construct a Naive Bayes classifier. We can write:

$$Pr\left(p|o,t,h,w\right)\propto Pr\left(p\right)Pr(o|p)Pr(t|p)Pr(h|p)Pr(w|p)$$

To calculate $Pr(p)$ we use marginal probablity.

In [5]:
def marginal_prob(df, column):
 '''
 Compute the marignal probability for values in a column
 '''
 # an array contain pairs of (value, count)
 vals_counts = [(val, (df[column] == val).sum()) for val in set(df[column])]
 print(vals_counts)
 total_count = sum([count for val, count in vals_counts])
 
 # an array contain pairs of (value, probability)
 vals_probs = [(val, count/total_count) for val, count in vals_counts]
 # a dictionary in which keys are val and values are the corresponding probabilities
 return dict(vals_probs)

print(marginal_prob(df,'outlook'))

print(set(df['outlook']))



[(0, 2), (1, 2), (2, 1)]
{0: 0.4, 1: 0.4, 2: 0.2}
{0, 1, 2}


To calculate probability of a feature given the class (play) we use conditinoal probability.

In [6]:
def conditional_prob(df, feature, c, val):
 # c is the class (0 or 1)
 df2 = df[df[c] == val][feature]
 vals_counts = [[val, (df2 == val).sum() + 1e-8] for val in set(df[feature])]
 total_count = sum([count for val, count in vals_counts])
 
 vals_probs = [(val, count/total_count) for val, count in vals_counts]
 return dict(vals_probs)


conditional_prob(df,'outlook','play',1)

{0: 0.49999999750000007, 1: 0.49999999750000007, 2: 4.999999925000001e-09}

Now we can use Bayes rule:

In [7]:
o = 1
t = 0
h = 0
w = 0

c = 0
p0 = marginal_prob(df, 'play')[c] * conditional_prob(df, 'outlook', 'play', c)[o] * conditional_prob(df, 'temp', 'play', c)[t] \
* conditional_prob(df, 'humid', 'play', c)[h] * conditional_prob(df, 'wind', 'play', c)[w]

c = 1
p1 = marginal_prob(df, 'play')[c] * conditional_prob(df, 'outlook', 'play', c)[o] * conditional_prob(df, 'temp', 'play', c)[t] \
* conditional_prob(df, 'humid', 'play', c)[h] * conditional_prob(df, 'wind', 'play', c)[w]

# normalizing
p_sum = p0 + p1
p0 /= p_sum
p1 /= p_sum

print("probability of not playing: {}".format(p0))
print("probability of playing : {}".format(p1))

[(0, 3), (1, 2)]
[(0, 3), (1, 2)]
probability of not playing: 0.0689655189536266
probability of playing : 0.9310344810463733


---
### Example 2

Suppose we have documents below as our training set. 

 d1: Chinese Beijing Chinese , class = C
 d2: Chinese Chinese Shanghai, class = C
 d3: Chinese Macao , class = C
 d4: Tokyo Japan Chinese , class = J


**Exercise:** Train a NB classifier and predict if `d5` belongs to class C or J.

 d5: Chinese Chinese Tokyo Japan, class = ?
 
What about `d6: Chinese Chinese Chinese Tokyo Japan, class = ?`

In [8]:
from collections import defaultdict
import math

In [9]:
# Training documents
docs = [
    {'words': ['Chinese', 'Beijing', 'Chinese'], 'class': 'C'},
    {'words': ['Chinese', 'Chinese', 'Shanghai'], 'class': 'C'},
    {'words': ['Chinese', 'Macao'], 'class': 'C'},
    {'words': ['Tokyo', 'Japan', 'Chinese'], 'class': 'J'}
]

In [10]:
# Count the number of documents in each class
class_count = defaultdict(int)
for doc in docs:
    class_count[doc['class']] += 1

In [11]:
# Calculate the prior probability of each class
class_prob = {}
total_count = len(docs)
for c in class_count:
    class_prob[c] = class_count[c] / total_count

In [12]:
# Count the number of occurrences of each word in each class
word_count = defaultdict(lambda: defaultdict(int))
for doc in docs:
    c = doc['class']
    for word in doc['words']:
        word_count[word][c] += 1

In [13]:
# Calculate the conditional probability of each word given each class
word_prob = defaultdict(lambda: defaultdict(float))
alpha = 1  # Laplace smoothing parameter
for word in word_count:
    total_c = sum(word_count[word].values())
    for c in class_count:
        word_prob[word][c] = (word_count[word][c] + alpha) / (total_c + alpha * len(word_count))

In [14]:
# Classify a new document
new_doc = {'words': ['Chinese', 'Chinese', 'Tokyo', 'Japan']}
doc_prob = defaultdict(float)
for c in class_prob:
    doc_prob[c] = math.log(class_prob[c])
    for word in new_doc['words']:
        if word in word_prob:
            doc_prob[c] += math.log(word_prob[word][c])
    # We can skip calculating P(d) because it's the same for all classes
predicted_class = max(doc_prob, key=doc_prob.get)
print("d5: Chinese Chinese Tokyo Japan, class = " + predicted_class)


d5: Chinese Chinese Tokyo Japan, class = C


In [15]:
new_doc = {'words': ['Chinese', 'Chinese', 'Chinese', 'Tokyo', 'Japan']}
doc_prob = defaultdict(float)
for c in class_prob:
    doc_prob[c] = math.log(class_prob[c])
    for word in new_doc['words']:
        if word in word_prob:
            doc_prob[c] += math.log(word_prob[word][c])
predicted_class = max(doc_prob, key=doc_prob.get)
print("d6: Chinese Chinese Chinese Tokyo Japan, class = " + predicted_class)

d6: Chinese Chinese Chinese Tokyo Japan, class = C
